<a href="https://colab.research.google.com/github/Magudeshgit/GloVe-Embeddings---Guessing-Game---/blob/master/Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim

1.) Get Dataset
2.) Reduce Dimension 200->50
3.) Create a AnnoyIndex
4.) Run Similiarity Test

In [4]:
import gensim.downloader as dn
word_vec = dn.load("glove-wiki-gigaword-300")

In [5]:
def generate_matrix():
    global vectorkeys
    vectorkeys = list(word_vec.index_to_key[:100000])
    matrix = []
    print("Initialzing Matrix Generation")
    for i,word in enumerate(vectorkeys):
      matrix.append(word_vec.get_vector(word))
    print("Matrix Generation Complete")
    return matrix


In [6]:
matrix_vecs = generate_matrix()

Initialzing Matrix Generation
Matrix Generation Complete


Reduction from 200dim to 50dim

In [7]:
from sklearn.decomposition import TruncatedSVD
k=TruncatedSVD(50)
reduced_matrix = k.fit_transform(matrix_vecs)

In [ ]:
reduced_matrix.shape

In [9]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553320 sha256=1c5456248503fee8cfff9556a8e72d21135febfad92af189a110855bfc91f400
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [10]:
from annoy import AnnoyIndex
Aindex = AnnoyIndex(50, 'angular')
word_to_vec = {}
for i, vector in enumerate(reduced_matrix):
  Aindex.add_item(i, vector)
  word_to_vec[vectorkeys[i]] = vector

Aindex.build(10)
Aindex.save('indexVector.ann')


True

In [11]:
word_to_vec['the'].shape


(50,)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import pprint
vc = word_vec.index_to_key[:100000]
vectorkeys = list(vc)
def fetch_similarities(word_vector):
    a=Aindex.get_nns_by_vector(word_vector, 5)
    b=cosine_similarity([word_vector], [reduced_matrix[i] for i in a])
    print(vectorkeys[a[b.argmax() + 1]])
    # print(vectorkeys[5],reduced_matrix[ind], vectorkeys[ind])

In [13]:
import numpy as np
def return_similars(word):
  a=Aindex.get_nns_by_vector(word_to_vec[word], 10)
  ls = []
  ls.append([vectorkeys[i] for i in a])
  ls[0].pop(0)
  return ls

In [14]:
a= return_similars('kind')

Actual Game Code

In [ ]:
import random
import pprint
common_word = [
    "apple", "river", "school", "car", "dog",
    "music", "house", "book", "city", "water",
    "tree", "computer", "light", "food", "phone",
    "family", "train", "chair", "friend", "money"
]
while True:
  print("Guess The Word Game!!!")
  print("Press Enter to start the game!!!")
  input()

  guess_word = common_word[random.randint(0, len(common_word)-1)]

  print("\n Guess the word")
  print(f'Your Hints are:\n')
  for i in return_similars(guess_word)[0]:
    print(i)
  guess = input("Enter your guess")
  if guess == guess_word:
    print("Voila, You won. You Guessed it right!!!")
  else:
    print(f"Nayy, You Lose. It was {guess_word}")

Guess The Word Game!!!
Press Enter to start the game!!!


 Guess the word
Your Hints are:

sells
selling
priced
menu
products
amp
buys
cheaper
brand
Enter your guesslk
Nayy, You Lose. It was apple
Guess The Word Game!!!
Press Enter to start the game!!!


 Guess the word
Your Hints are:

cities
outside
where
central
area
campus
site
hometown
district
Enter your guesslk
Nayy, You Lose. It was city
Guess The Word Game!!!
Press Enter to start the game!!!
